# Import packages

In [77]:
####import packages
using FFTW
using Distributions
#using NBInclude
#using Plots
using PyCall
using PyPlot
#using WebIO
using Pkg
using Interpolations
using LinearAlgebra
using Profile
using TensorOperations
import NaNMath
using DelimitedFiles
using Hilbert

#get to the root of the project to ease path later on
if pwd()[end-7:end] == "Analysis"
    cd("..")
end

###import local files (CAUTION: kernel must be restarted if changes are made)
include("src/Coupling.jl") 
include("src/CouplingOptimization.jl")
include("src/Phase.jl") 
include("src/PhaseSim.jl") 
include("src/URTS.jl")
include("src/HMM.jl") 
include("src/SignalAnalysis.jl") 

#@nbinclude("Types.ipynb")

###set nice plotting style
@pyimport matplotlib.pyplot as p
p.style[:use]("seaborn-whitegrid")



# Get the dict of traces and inferred parameters

In [78]:
d_all = Dict()
for namedir in readdir("Data")
    if namedir==".DS_Store"
        continue
    end
    
    #get list of real parameters
    l_param_true = readdlm("Data/"*namedir*"/Parameters/original.txt")
    
    #get list of inferred parameters
    l_param_inf = readdlm("Data/"*namedir*"/Parameters/inferred.txt")
    w_theta_inf = readdlm("Data/"*namedir*"/Parameters/inferred_w_theta.txt")
    w_phi_inf = readdlm("Data/"*namedir*"/Parameters/inferred_w_phi.txt")
    
    #remove useless dimensions
    w_theta_inf = dropdims(w_theta_inf; dims=2)
    w_phi_inf = dropdims(w_phi_inf; dims=2)
    
    
    #replace the string by the real waveforms in l_param_inf
    l_param_inf[1] = w_theta_inf
    l_param_inf[2] = w_phi_inf
    
    #get traces
    l_traces = []
    for name_idx_trace in readdir("Data/"*namedir*"/Traces")
        trace = readdlm("Data/"*namedir*"/Traces/"*name_idx_trace)
        push!(l_traces,trace)
    end
    
    #add traces and parameters to dictionnary
    d_all[l_param_true] = (l_traces, l_param_inf)
end   

## Optimize coupling

In [3]:
function opt(;N_iter = 10, n_tr = 100, resolution = 50,  α = 0.01, β = 1., κ = 0., lambd = 2, Theta = Theta, Phi = Phi, do_plot = false)
    #initial guess for coupling is white noise
    F_theta = zeros(resolution, resolution) #(rand( resolution,resolution).-0.5).*0.001
    F_phi = zeros(resolution, resolution)  #(rand( resolution,resolution).-0.5).*0.001
    l_ll = []
    for it_opt in 1:N_iter
                
        FT_θ =  fft(F_theta)#Theta.FT_F   #
        FT_ϕ =  fft(F_phi)#Phi.FT_F     #
                
        #reset hidden variables (to recompute coupling)
        reset_F(Theta)
        reset_F(Phi)
        
        l_res, l_ll = compute_phases(FT_θ, FT_ϕ, n_tr,  α , β, κ )
        l_x = [res[1] for res in l_res]
        l_p = [res[2] for res in l_res]
        l_jP_x = [res[3] for res in l_res]
        l_jP_P = [res[4] for res in l_res]

        #println(l_p[1])
        
        F_theta, F_phi, F_theta_bad, F_phi_bad, Mat_norm, FT_θ, FT_ϕ = compute_coupling_discretization(l_x, l_p, l_jP_x, l_jP_P, resolution, lambd)
        #FT_θ, FT_ϕ = compute_coupling_fourier( FT_θ, FT_ϕ, size(FT_θ,1) , l_x, l_p, l_jP_x, l_jP_P, α, β, κ, lambd, Theta.f, Phi.f, dt )
        
        F_theta = real.(ifft(FT_θ)) 
        F_phi = real.(ifft(FT_ϕ ))
        
        if do_plot
            #println("Iteration ", it_opt, " done")
            #plot 
            plot_F(F_theta, interpolation = "none", title_fig = "Coupling_θ_prob", save = true)
            plot_F(F_theta_bad, interpolation = "none", title_fig = "Coupling_θ_bad", save = true)

            plot_F(F_phi, interpolation = "none", title_fig = "Coupling_ϕ_prob", save = true)
            plot_F(F_phi_bad, interpolation = "none", title_fig = "Coupling_ϕ_bad", save = true)

            #plot_F(Mat_norm, vmin = 0, vmax = 0.5)
        end
                
        #println("LL : ", sum(l_ll))
    end

    return F_theta, F_phi, sum(l_ll)
end


function opt_sigma_parameters(;n_tr = 100, resolution = 20,  lambd = 2, Theta = Theta, Phi = Phi)
    ls = Array{Float64}(undef, 0, 5,)
    s = 0
    ll = 0
    
    F_θ =  ifft(Theta.FT_F)
    F_ϕ =  ifft(Phi.FT_F )
    
    
    for alpha in 0.01:0.2:2
        for beta in 0:0.2:2
            for kappa in 0:0.2:2
                try
                    F_theta, F_phi, ll = opt(N_iter = 1, n_tr = n_tr, resolution = resolution,  α = alpha, β = beta, κ = kappa, lambd = lambd, Theta = Theta, Phi = Phi)
                    s = NaNMath.sum(abs.(F_theta-F_θ)+abs.(F_phi-F_ϕ))
                catch
                    s = 100000000
                    ll = -1000000000
                end
                    
                
                ls = vcat(ls, [alpha beta kappa s ll])
                println(alpha, beta, kappa, s, ll)
            end
        end
    end
    return ls
end
        

        
opt(N_iter = 1, n_tr = 100, resolution = 50, do_plot = true)#,  α = 1.81, β = 0., κ = 0.6)
#ls = opt_sigma_parameters(n_tr = 10, resolution = 30,  lambd = 2, Theta = Theta, Phi = Phi)        


UndefVarError: UndefVarError: Theta not defined

In [4]:
#keep only best fits, and the looked for the less biased ones
best_ll_ls = sortslices(ls, dims = 1, by=x->(x[5]))[end-100:end,:]
less_biased_ls = sortslices(best_ll_ls, dims = 1, by=x->(x[4]))[1:10,:]



UndefVarError: UndefVarError: ls not defined

In [5]:
#try the combination 1.81  0.0  0.6
#opt(N_iter = 1, n_tr = 100, resolution = 20, α = 1.81, β = 0., κ = 0.6, do_plot = true)

In [6]:
#@profile opt(1, 20, 50)
open("prof.txt", "w") do s
    Profile.print(IOContext(s, :displaysize => (5000, 5000)), format=:flat)
end

┌ Warning: There were no samples collected. Run your program longer (perhaps by
│ running it multiple times), or adjust the delay between samples with
│ `Profile.init()`.
└ @ Profile /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.0/Profile/src/Profile.jl:659


In [7]:
x = [1 , 2 , 3]
println(x)

[1, 2, 3]
